In [3]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import cv2
from skimage.feature import hog

from sklearn import svm
from sklearn.model_selection import train_test_split

c:\Users\victo\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Lendo dados

In [72]:
def process_image(image_path):
    # Abrir a imagem e convertê-la para escala de cinza
    img = Image.open(image_path).convert('L')
    # Converter a imagem em um array numpy
    img_array = np.array(img)
    # Binarizar a imagem (0 para preto e 1 para branco)
    # img_binary = (img_array > 127).astype(int)  # Usando um limiar de 127
    return img_array

In [73]:
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg'):
            img_path = os.path.join(folder_path, filename)
            img_binary = process_image(img_path)
            images.append(img_binary.flatten())
    return images

In [74]:
def load_strings_from_folder(folder_path):
    strings = []
    filenames = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r') as file:
                    content = file.read().strip()
                    strings.append(content)
                    filenames.append(filename)
            except Exception as e:
                print(f"Erro ao ler o arquivo {file_path}: {e}")
    return strings, filenames

In [223]:
treinamento = load_images_from_folder('dados/CAPTCHA-10k/treinamento')
treinamento = np.array(treinamento)
validacao = load_images_from_folder('dados/CAPTCHA-10k/validacao')
validacao = np.array(validacao)
teste = load_images_from_folder('dados/CAPTCHA-10k/teste')
teste = np.array(teste)
labels = load_strings_from_folder('dados/CAPTCHA-10k/labels10k')[0]

In [76]:
treino_x = treinamento
treino_y = labels[:8000]

validacao_x = validacao
validacao_y = labels[8001:9000]

teste_x = teste
teste_y = labels[9001:]

In [1]:
def print_img(image):
    plt.imshow(image, cmap='gray', interpolation='nearest')
    plt.axis('off')
    plt.show()

In [154]:
def apply_unsharp_mask(image, kernel_size=(5, 5), sigma=1.5):
    kernel = np.array([[0, -1, 0],
                       [-1, 6,-1],
                       [0, -1, 0]])
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    sharpened = cv2.filter2D(blurred, -1, kernel)
    
    return sharpened

In [201]:
def particionar_img(image_array):
    image_array = np.reshape(image_array, (50, 180))
    # print_img(image_array)
    particoes = []
    offset = 30
    for i in range(0, 6):
        particao = image_array[10: 45, 5+offset*i:45+offset*i]
        particoes.append(particao)
        particao = apply_unsharp_mask(particao)
        # print_img(particao)
    particoes[5] = np.pad(particoes[5], ((0, 0), (7, 8)), 'constant', constant_values=255)
    return particoes

In [207]:
def aplicar_particao(images):
    images_particionadas = []
    for i in range(0, len(images)):
        images_particionadas.append(particionar_img(images[i]))
    return np.array(images_particionadas)

In [224]:
def particonar_strings(strings):
    strings_particionadas = []
    for i in range(0, len(strings)):
        string = []
        for j in range(0, 6):
            string.append(strings[i][j])
        
        strings_particionadas.append(string)
    
    return np.array(strings_particionadas)

# HOG

In [58]:
def preprocess_image(image_array):
    # Certifique-se de que a imagem está no formato correto (180x50)
    image_array = np.reshape(image_array, (50, 180))
    
    # Calcular os recursos HOG diretamente do array
    fd, hog_image = hog(image_array, orientations=64, pixels_per_cell=(50, 30),
                        cells_per_block=(1, 1), visualize=True)
    

    return fd, hog_image

# CNN